In [1]:
import pandas as pd 

In [5]:
train_meta_df = pd.read_parquet('./data/train_meta.parquet', engine='fastparquet')
train_meta_df

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117
...,...,...,...,...,...,...
131953919,660,2147483597,24339210,24339349,5.895612,0.333071
131953920,660,2147483603,24339350,24339383,3.273695,1.503301
131953921,660,2147483617,24339384,24339424,2.945376,1.723253
131953922,660,2147483626,24339425,24339499,1.616582,1.937025


In [3]:
batch_1_df = pd.read_parquet('./data/train/batch_1.parquet', engine='fastparquet')
batch_1_df

,sensor_id,time,charge,auxiliary
event_id,,,,
24,3918,5928,1.325,True
24,4157,6115,1.175,True
24,3520,6492,0.925,True
24,5041,6665,0.225,True
24,2948,8054,1.575,True
...,...,...,...,...
3266196,1961,17617,0.975,True
3266196,36,17705,0.625,True
3266196,356,18192,0.725,True
